In [ ]:
import numpy as np
import os
from skimage.io import imread,imsave
from utils.maps_creation import calculate_sc,calculate_smooth
import cv2
from utils import extract_utils

dir = '/datadisk/datasets/breast/expert'
expert_names = ['NP2','NP6','NP8','NP14','NP18','NP20']
files = []
for expert in expert_names:
    files.extend(os.listdir(f'{dir}/masks/Train/{expert}'))
files = np.unique(files)

In [ ]:
remove = ['A0BG_BH', 'A0DA_A7', 'A0SK_A1', 'A0YM_A2', 'A1AY_AR', 'A1LS_E2',
       'A26Y_C8', 'A5D6_OL', 'A66I_OL', 'A73Y_LL']
count = 0
for i,file in enumerate(files):

    file_split = file.split('_')
    name = file_split[1]+'_'+file_split[2]
    if name in remove:
        print(f'{i}: {file}')
        count+=1
        file_name = file.split('.')[0]
        weights = []
        experts = []
        for exp in expert_names:
            if os.path.isfile(f'{dir}/masks/Train/{exp}/{file}'):
                expert= imread(f'{dir}/masks/Train/{exp}/{file}')
                expert[expert==5]=0
                expert[expert==4]=0
                experts.append(expert)

        
        print('num experts ',len(experts))

        _,_,smooth_final = calculate_smooth(experts,num_cl=4,size=512)
        _,_,sc_final = calculate_sc(experts,weights,num_cl=4,size=512)
        np.save(f'{dir}/masks/Train/sc_v3/{file_name}.npy',sc_final)
        np.save(f'{dir}/masks/Train/smooth_v2/{file_name}.npy',smooth_final)

2462: core_A5D6_OL_x_ini_0_y_ini_0.png
num experts  1
2463: core_A5D6_OL_x_ini_0_y_ini_1074.png
num experts  1
2464: core_A5D6_OL_x_ini_0_y_ini_1432.png
num experts  1
2465: core_A5D6_OL_x_ini_0_y_ini_1790.png
num experts  1
2466: core_A5D6_OL_x_ini_0_y_ini_2148.png
num experts  1
2467: core_A5D6_OL_x_ini_0_y_ini_358.png
num experts  1
2468: core_A5D6_OL_x_ini_0_y_ini_716.png
num experts  1
2469: core_A5D6_OL_x_ini_1074_y_ini_0.png
num experts  1
2470: core_A5D6_OL_x_ini_1074_y_ini_1074.png
num experts  1
2471: core_A5D6_OL_x_ini_1074_y_ini_1432.png
num experts  1
2472: core_A5D6_OL_x_ini_1074_y_ini_1790.png
num experts  1
2473: core_A5D6_OL_x_ini_1074_y_ini_2148.png
num experts  1
2474: core_A5D6_OL_x_ini_1074_y_ini_358.png
num experts  1
2475: core_A5D6_OL_x_ini_1074_y_ini_716.png
num experts  1
2476: core_A5D6_OL_x_ini_1432_y_ini_0.png
num experts  1
2477: core_A5D6_OL_x_ini_1432_y_ini_1074.png
num experts  1
2478: core_A5D6_OL_x_ini_1432_y_ini_1432.png
num experts  1
2479: core_A5D

In [ ]:
for i,file in enumerate(files):
    file_name = file.split('.')[0]
    print(f'{i}: {file}')
    experts = []
    for exp in range(len(expert_names)):
        experts.append(imread(f'{dir}expert_masks/{exp+1}/{file}'))

    _,_,smooth_final = calculate_smooth(experts)
    _,_,sc_final = calculate_sc(experts,weights)
    expert_mask = imread(f'{dir}train_masks/{file}')
    # sc_or_map = sc_expert_map(expert_mask,weights)
    # sc_final = soft(torch.from_numpy(sc_final)).cpu().detach().numpy()
    im = imread(f'{dir}train_imgs/{file}')

    im1,im2,im3 = cv2.split(im.astype('float'))
    PATCHSIZE = 512 ##  try 256, 512

    img_mask = cv2.merge((im1,im2,im3,expert_mask.astype('float')))
    for i in range(6):
        img_mask = cv2.merge((img_mask,smooth_final[i]))

    for i in range(6):
        img_mask = cv2.merge((img_mask,sc_final[i]))
    # img_mask = cv2.merge((img_mask,smooth_final))  


    patches = extract_utils.extract_patches(img_mask, PATCHSIZE, [PATCHSIZE,PATCHSIZE])

    count = 0
    for patch in patches:
        cl,counts = np.unique(patch[:,:,3],return_counts=True)
        if 0 in cl:
            if counts[0]/PATCHSIZE**2 <= 0.2:
                imsave(f'{dir}/patches/masks/expert/{file_name}_{count}.png',patch[:,:,3].astype('uint8'))
                imsave(f'{dir}/patches/imgs/{file_name}_{count}.png',patch[:,:,:3].astype('uint8'))
                np.save(f'{dir}/patches/masks/smooth/{file_name}_{count}.npy',patch[:,:,4:10])
                np.save(f'{dir}/patches/masks/sc/{file_name}_{count}.npy',patch[:,:,10:])
        else:
            imsave(f'{dir}/patches/masks/expert/{file_name}_{count}.png',patch[:,:,3].astype('uint8'))
            imsave(f'{dir}/patches/imgs/{file_name}_{count}.png',patch[:,:,:3].astype('uint8'))
            np.save(f'{dir}/patches/masks/smooth/{file_name}_{count}.npy',patch[:,:,4:10])
            np.save(f'{dir}/patches/masks/sc/{file_name}_{count}.npy',patch[:,:,10:])
        count+=1



0: fdb9d38ce2b5a1d31bb030cd2d3a03b9_0.png


/home/laura/anaconda3/envs/multi/lib/python3.12/site-packages/torch/nn/modules/module.py:1736: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)
/home/laura/anaconda3/envs/multi/lib/python3.12/site-packages/skimage/_shared/utils.py:328: UserWarning: /home/laura/Documents/dataset/PANDA//patches/masks/expert/fdb9d38ce2b5a1d31bb030cd2d3a03b9_0_3.png is a low contrast image
  return func(*args, **kwargs)
/home/laura/anaconda3/envs/multi/lib/python3.12/site-packages/skimage/_shared/utils.py:328: UserWarning: /home/laura/Documents/dataset/PANDA//patches/masks/expert/fdb9d38ce2b5a1d31bb030cd2d3a03b9_0_9.png is a low contrast image
  return func(*args, **kwargs)
/home/laura/anaconda3/envs/multi/lib/python3.12/site-packages/skimage/_shared/utils.py:328: UserWarning: /home/laura/Documents/dataset/PANDA//patches/masks/expert/fdb9d38ce2b5a1d31bb030cd2d3a03b9_0_10.png is a low contrast i

1: d4dcb1cd15053d17de19d2d63ddcb173_2.png


/home/laura/anaconda3/envs/multi/lib/python3.12/site-packages/torch/nn/modules/module.py:1736: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)
/home/laura/anaconda3/envs/multi/lib/python3.12/site-packages/skimage/_shared/utils.py:328: UserWarning: /home/laura/Documents/dataset/PANDA//patches/masks/expert/d4dcb1cd15053d17de19d2d63ddcb173_2_0.png is a low contrast image
  return func(*args, **kwargs)
/home/laura/anaconda3/envs/multi/lib/python3.12/site-packages/skimage/_shared/utils.py:328: UserWarning: /home/laura/Documents/dataset/PANDA//patches/masks/expert/d4dcb1cd15053d17de19d2d63ddcb173_2_1.png is a low contrast image
  return func(*args, **kwargs)
/home/laura/anaconda3/envs/multi/lib/python3.12/site-packages/skimage/_shared/utils.py:328: UserWarning: /home/laura/Documents/dataset/PANDA//patches/masks/expert/d4dcb1cd15053d17de19d2d63ddcb173_2_2.png is a low contrast im

2: b87d20bcf0c7492853ca6cd000e39666_0.png


...

/home/laura/anaconda3/envs/multi/lib/python3.12/site-packages/torch/nn/modules/module.py:1736: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)
/home/laura/anaconda3/envs/multi/lib/python3.12/site-packages/skimage/_shared/utils.py:328: UserWarning: /home/laura/Documents/dataset/PANDA//patches/masks/expert/ce7817cab3ef2a86003458ddab80269b_7_7.png is a low contrast image
  return func(*args, **kwargs)
/home/laura/anaconda3/envs/multi/lib/python3.12/site-packages/skimage/_shared/utils.py:328: UserWarning: /home/laura/Documents/dataset/PANDA//patches/masks/expert/ce7817cab3ef2a86003458ddab80269b_7_13.png is a low contrast image
  return func(*args, **kwargs)
/home/laura/anaconda3/envs/multi/lib/python3.12/site-packages/skimage/_shared/utils.py:328: UserWarning: /home/laura/Documents/dataset/PANDA//patches/masks/expert/ce7817cab3ef2a86003458ddab80269b_7_14.png is a low contrast 

88: e62a7175ca466fb365631d124972c0a8_3.png


/home/laura/anaconda3/envs/multi/lib/python3.12/site-packages/torch/nn/modules/module.py:1736: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)
/home/laura/anaconda3/envs/multi/lib/python3.12/site-packages/skimage/_shared/utils.py:328: UserWarning: /home/laura/Documents/dataset/PANDA//patches/masks/expert/e62a7175ca466fb365631d124972c0a8_3_0.png is a low contrast image
  return func(*args, **kwargs)
/home/laura/anaconda3/envs/multi/lib/python3.12/site-packages/skimage/_shared/utils.py:328: UserWarning: /home/laura/Documents/dataset/PANDA//patches/masks/expert/e62a7175ca466fb365631d124972c0a8_3_1.png is a low contrast image
  return func(*args, **kwargs)
/home/laura/anaconda3/envs/multi/lib/python3.12/site-packages/skimage/_shared/utils.py:328: UserWarning: /home/laura/Documents/dataset/PANDA//patches/masks/expert/e62a7175ca466fb365631d124972c0a8_3_2.png is a low contrast im

89: 72e996bf235d7bcec83fd9c8f7de1977_3.png


/home/laura/anaconda3/envs/multi/lib/python3.12/site-packages/torch/nn/modules/module.py:1736: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)
/home/laura/anaconda3/envs/multi/lib/python3.12/site-packages/skimage/_shared/utils.py:328: UserWarning: /home/laura/Documents/dataset/PANDA//patches/masks/expert/72e996bf235d7bcec83fd9c8f7de1977_3_4.png is a low contrast image
  return func(*args, **kwargs)
/home/laura/anaconda3/envs/multi/lib/python3.12/site-packages/skimage/_shared/utils.py:328: UserWarning: /home/laura/Documents/dataset/PANDA//patches/masks/expert/72e996bf235d7bcec83fd9c8f7de1977_3_5.png is a low contrast image
  return func(*args, **kwargs)
/home/laura/anaconda3/envs/multi/lib/python3.12/site-packages/skimage/_shared/utils.py:328: UserWarning: /home/laura/Documents/dataset/PANDA//patches/masks/expert/72e996bf235d7bcec83fd9c8f7de1977_3_10.png is a low contrast i